# **K-Means Clustering**

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import math
import random

data = pd.read_csv('../input/dataset/data.csv')
label = pd.read_csv('../input/labeld/label.csv')

In [2]:
data.head(10)

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
label.head(10)

,7
0,2
1,1
2,0
3,4
4,1
5,4
6,9
7,5
8,9
9,0


In [4]:
data.shape

(9999, 784)

In [5]:
label.shape

(9999, 1)

In [6]:
data.describe()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
count,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,...,9999.000000,9999.000000,9999.000000,9999.000000,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.179318,0.163616,0.052605,0.000600,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.674433,5.736359,2.420125,0.060003,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,253.000000,253.000000,156.000000,6.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
label.describe()

,7
count,9999.000000
mean,4.443144
std,2.895897
min,0.000000
25%,2.000000
50%,4.000000
75%,7.000000
max,9.000000


In [8]:
def euclidean_dist (pcentroids, data):
    edist = np.zeros((len(pcentroids),len(data)))
    for i in range(len(pcentroids)):
        euc_dist = []
        for point in data:
            euc_dist.append(np.sqrt(np.sum(np.square(point-pcentroids[i]))))
        edist[i] = euc_dist
    return edist

def cosine_dist (pcentroids, data):
    cdist = np.zeros((len(pcentroids),len(data)))
    for i in range(len(pcentroids)):
        cos_dist = []
        for point in data:
            cos_dist.append(1 - (np.dot(pcentroids[i],point)/(np.linalg.norm(pcentroids[i])*np.linalg.norm(point))))
        cdist[i] = cos_dist
    return cdist
def jaccard_dist (pcentroids, data):
    jdist = np.zeros((len(pcentroids),len(data)))
    for i in range(len(pcentroids)):
        jac_dist = []
        for point in data:
            min_arr = np.minimum(pcentroids[i], point)
            max_arr = np.maximum(pcentroids[i], point)
            jac_dist.append(1-(np.sum(min_arr)/np.sum(max_arr)))
        jdist[i] = jac_dist
    return jdist

In [9]:
np.random.seed(123)

def stop_conditions(iter, centroids, old_centroids):
    if np.array_equal(old_centroids, centroids) or iter == 100:
        return False
    return True

def find_labels (dist, centroids, data):
    labels = np.array([])
    for i in range(len(data)):
        index = np.where(dist == np.min(dist[:,i]))
        labels = np.append(labels, index[0][0])
    return labels

def calculate_centroid(data, labels, pcentroids, k, features):
    ncentroids = np.zeros((k, features))
    temp = [0]*k
    for i in range(len(data)):
        temp[labels[i]] += 1
        ncentroids[labels[i]] += data[i]
    for i in range(k):
        if temp[i] != 0:
            ncentroids[i] = ncentroids[i]/temp[i]
        else:
            ncentroids[i] = pcentroids[i]
    return ncentroids

def K_Means (data, k_centroids, dist_method, stop_criteria, flag):
    
    features = len(data[0])
    cur_centroids = np.zeros((k_centroids, features))
    for i in range(k_centroids):
        r = np.random.randint(0, features+1)
        cur_centroids[i, :] = data[r]
    
    current_iter = 0
    pcentroids = np.zeros((k_centroids, features))
    labels = None
    old_sse = np.Inf
    sse = np.Inf
    
    if (stop_criteria == 1):
        while stop_conditions(current_iter, cur_centroids, pcentroids):
            pcentroids = cur_centroids
            current_iter += 1
            if (dist_method == 'euclidean'):
                dist = euclidean_dist(pcentroids, data)
            elif (dist_method == 'cosine'):
                dist = cosine_dist(pcentroids, data)
            elif (dist_method == 'jaccard'):
                dist = jaccard_dist(pcentroids, data)
            else:
                print('Wrong distance name')
                return
            labels = find_labels(dist, pcentroids, data)
            cur_centroids = calculate_centroid(data, labels.astype(int), pcentroids, k_centroids, features)
    elif (stop_criteria == 2):
        while stop_conditions_2(current_iter, cur_centroids, pcentroids, old_sse, sse):
            old_sse = sse
            pcentroids = cur_centroids
            current_iter += 1
            if (dist_method == 'euclidean'):
                dist = euclidean_dist(pcentroids, data)
            elif (dist_method == 'cosine'):
                dist = cosine_dist(pcentroids, data)
            elif (dist_method == 'jaccard'):
                dist = jaccard_dist(pcentroids, data)
            else:
                print('Wrong distance name')
                return
            labels = find_labels(dist, pcentroids, data)
            cur_centroids = calculate_centroid(data, labels.astype(int), pcentroids, k_centroids, features)
            sse = SSE(cur_centroids, labels.astype(int), data)
    else:
        while stop_conditions_3(current_iter, cur_centroids, pcentroids, old_sse, sse, flag):
            old_sse = sse
            pcentroids = cur_centroids
            current_iter += 1
            if (dist_method == 'euclidean'):
                dist = euclidean_dist(pcentroids, data)
            elif (dist_method == 'cosine'):
                dist = cosine_dist(pcentroids, data)
            elif (dist_method == 'jaccard'):
                dist = jaccard_dist(pcentroids, data)
            else:
                print('Wrong distance name')
                return
            labels = find_labels(dist, pcentroids, data)
            cur_centroids = calculate_centroid(data, labels.astype(int), pcentroids, k_centroids, features)
            sse = SSE(cur_centroids, labels.astype(int), data)
    print(dist_method, ' ', 'Iterations total : ', current_iter)
    return cur_centroids, labels
        
            


In [10]:

E_centroids, E_labels = K_Means(data.to_numpy(), 10, 'euclidean', 1, 1)
C_centroids, C_labels = K_Means(data.to_numpy(), 10, 'cosine', 1, 1)
J_centroids, J_labels = K_Means(data.to_numpy(), 10, 'jaccard', 1, 1)

euclidean   Iterations total :  71
cosine   Iterations total :  85
jaccard   Iterations total :  45


In [11]:
def SSE (centroids, labels, data):
    x = 0
    for i in range(len(data)):
        index = labels[i]
        x += np.sum(np.square(data[i]-centroids[index]))
    return x
E_SSE = SSE (E_centroids, E_labels.astype(int), data.to_numpy())
C_SSE = SSE (C_centroids, C_labels.astype(int), data.to_numpy())
J_SSE = SSE (J_centroids, J_labels.astype(int), data.to_numpy())

print('SSE of Euclidean K-Means: ', E_SSE)
print('SSE of Cosine K-Means: ', C_SSE)
print('SSE of Jaccard K-Means: ', J_SSE)
        

SSE of Euclidean K-Means:  25318301268.749744
SSE of Cosine K-Means:  25411581642.345028
SSE of Jaccard K-Means:  25483177207.831245


In [12]:
SSE_table = pd.DataFrame({
    'Similarity metrics': ['Euclidean K-Means', 'Cosine K-Means', 'Jaccard K-Means'],
    'SSE': [E_SSE, C_SSE, J_SSE],
})
SSE_table.sort_values(by='SSE', ascending=True)

,Similarity metrics,SSE
0,Euclidean K-Means,2.531830e+10
1,Cosine K-Means,2.541158e+10
2,Jaccard K-Means,2.548318e+10


## Evaluating the SSE values

Based on the following SSE values obtained during testing:
- Euclidean SSE:  25318301268.749744
- Cosine SSE:  25411581642.345028
- jaccard SSE:  25483177207.831245

It would seem that the Euclidean method has the least error since it had the lowest SSE score.

# Q2
## Comparing the accuracies

In [13]:
def find_accuracy (centroids, centroid_labels, original_label):
    label_map = {}
    x = 0
    for i in range(len(centroid_labels)):
        if centroid_labels[i] in label_map:
            label_map[centroid_labels[i]] = np.append(label_map[centroid_labels[i]], original_label['7'][i])
        else: 
            label_map[centroid_labels[i]] = np.array([original_label['7'][i]])
    for i in range(len(centroids)):
        if i in label_map:
            vote = np.bincount(label_map[i]).argmax()
            x += np.count_nonzero(label_map[i] == vote)
    return x/len(original_label)

E_accuracy = find_accuracy(E_centroids, E_labels, label)
C_accuracy = find_accuracy(C_centroids, C_labels, label)
J_accuracy = find_accuracy(J_centroids, J_labels, label)

print ('Euclidean K-Means Accuracy: ', E_accuracy)
print('Cosine K-Means Accuracy: ', C_accuracy)
print('Jaccard K-Means Accuracy: ', J_accuracy)

Euclidean K-Means Accuracy:  0.5942594259425943
Cosine K-Means Accuracy:  0.6140614061406141
Jaccard K-Means Accuracy:  0.6233623362336234


In [14]:
accuracy_table = pd.DataFrame({
    'Similarity metrics': ['Euclidean K-Means', 'Cosine K-Means', 'Jaccard K-Means'],
    'Accuracy': [E_accuracy, C_accuracy, J_accuracy],
})
accuracy_table.sort_values(by='Accuracy', ascending=False)

,Similarity metrics,Accuracy
2,Jaccard K-Means,0.623362
1,Cosine K-Means,0.614061
0,Euclidean K-Means,0.594259


## Analyzing accuracies

Based on the following accuracy scores obtained through testing:
- Euclidean Accuracy:  0.5942594259425943
- Cosine Accuracy:  0.6140614061406141
- jaccard Accuracy:  0.6233623362336234

It would seem that the jaccard is the most accurate model based on the results of testing

# Q3
## Modifiying stop criteria to include SSE

In [15]:
def stop_conditions_2 (iter, centroids, old_centroids, old_sse, sse):
    if (iter != 0):
        if np.array_equal(old_centroids, centroids) or iter == 100 or sse >= old_sse:
            return False
    return True
E_centroids, E_labels = K_Means(data.to_numpy(), 10, 'euclidean', 2, 1)
C_centroids, C_labels = K_Means(data.to_numpy(), 10, 'cosine', 2, 1)
J_centroids, J_labels = K_Means(data.to_numpy(), 10, 'jaccard', 2, 1)



euclidean   Iterations total :  93
cosine   Iterations total :  27
jaccard   Iterations total :  38


## Comparing iters/times of models with modified stop criteria
Based on the results obtained during testing:
- iters till convergence:
    - Euclidean: 93
    - Cosine: 27
    - jaccard: 38
- Times: Each function runs around the same amount of time per iter so the shortest time was Cosine followed by jaccard and lastly the slowest was euclidean due to the massive number of iters this attempt took to converge

# Q4
## Modify stop conditions to consider only one terminating condition

In [16]:
def stop_conditions_3 (iter, centroids, old_centroids, old_sse, sse, flag):
    if (iter != 0):
        if (flag == 1):
            if np.array_equal(old_centroids, centroids):
                return False
        elif (flag == 2):
            if (sse >= old_sse):
                return False
        elif (flag == 3):
            if (iter == 100):
                return False
    return True


E_centroids_pos, E_labels_pos = K_Means(data.to_numpy(), 10, 'euclidean', 3, 1)
E_centroids_sse, E_labels_sse = K_Means(data.to_numpy(), 10, 'euclidean', 3, 2)
E_centroids_iter, E_labels_iter = K_Means(data.to_numpy(), 10, 'euclidean', 3, 3)
 
C_centroids_pos, C_labels_pos = K_Means(data.to_numpy(), 10, 'cosine', 3, 1)
C_centroids_sse, C_labels_sse = K_Means(data.to_numpy(), 10, 'cosine', 3, 2)
C_centroids_iter, C_labels_iter = K_Means(data.to_numpy(), 10, 'cosine', 3, 3)


J_centroids_pos, J_labels_pos = K_Means(data.to_numpy(), 10, 'jaccard', 3, 1)
J_centroids_sse, J_labels_sse = K_Means(data.to_numpy(), 10, 'jaccard', 3, 2)
J_centroids_iter, J_labels_iter = K_Means(data.to_numpy(), 10, 'jaccard', 3, 3)


euclidean   Iterations total :  38
euclidean   Iterations total :  36
euclidean   Iterations total :  100
cosine   Iterations total :  59
cosine   Iterations total :  53
cosine   Iterations total :  100
jaccard   Iterations total :  60
jaccard   Iterations total :  47
jaccard   Iterations total :  100


## Calculate SSE for each model

In [17]:
# Euclidean dist model SSEs
E_SSE_pos = SSE (E_centroids_pos, E_labels_pos.astype(int), data.to_numpy())
E_SSE_sse = SSE (E_centroids_sse, E_labels_sse.astype(int), data.to_numpy())
E_SSE_iter = SSE (E_centroids_iter, E_labels_iter.astype(int), data.to_numpy())

# Cosine dist model SSEs
C_SSE_pos = SSE (C_centroids_pos, C_labels_pos.astype(int), data.to_numpy())
C_SSE_sse = SSE (C_centroids_sse, C_labels_sse.astype(int), data.to_numpy())
C_SSE_iter = SSE (C_centroids_iter, C_labels_iter.astype(int), data.to_numpy())

# jaccard dist model SSEs
J_SSE_pos = SSE (J_centroids_pos, J_labels_pos.astype(int), data.to_numpy())
J_SSE_sse = SSE (J_centroids_sse, J_labels_sse.astype(int), data.to_numpy())
J_SSE_iter = SSE (J_centroids_iter, J_labels_iter.astype(int), data.to_numpy())

print('Euclidean SSEs wrt stop criteria: ')
print ('Centroids converge: ', E_SSE_pos)
print('New SSE is larger than previous SSE: ', E_SSE_sse)
print('Reached max number of iters (100): ', E_SSE_iter)

print('Cosine SSEs wrt stop criteria: ')
print ('Centroids converge: ', C_SSE_pos)
print('New SSE is larger than previous SSE: ', C_SSE_sse)
print('Reached max number of iters (100): ', C_SSE_iter)

print('jaccard SSEs wrt stop criteria: ')
print ('Centroids converge: ', J_SSE_pos)
print('New SSE is larger than previous SSE: ', J_SSE_sse)
print('Reached max number of iters (100): ', J_SSE_iter)

Euclidean SSEs wrt stop criteria: 
Centroids converge:  25321994318.21326
New SSE is larger than previous SSE:  25495249192.75157
Reached max number of iters (100):  25404775206.395275
Cosine SSEs wrt stop criteria: 
Centroids converge:  25468173815.67823
New SSE is larger than previous SSE:  25425454489.886997
Reached max number of iters (100):  25416733862.67944
jaccard SSEs wrt stop criteria: 
Centroids converge:  25415230917.736603
New SSE is larger than previous SSE:  25434179158.75361
Reached max number of iters (100):  25587905239.143787


In [18]:
newSSE_table = pd.DataFrame({
    'Similarity metrics': ['Euclidean SSE', 'Cosine SSE', 'Jaccard SSE'],
    'Centroids Convergence': [E_SSE_pos, C_SSE_pos, J_SSE_pos],
    'Previous SSE': [E_SSE, C_SSE, J_SSE],
    'New SSE': [E_SSE_sse, C_SSE_sse, J_SSE_sse],
    'Max Iterations': [E_SSE_iter, C_SSE_iter, J_SSE_iter],
})
print("Similarity metric with Best Centroid Convergence : ")
newSSE_table.sort_values(by='Centroids Convergence', ascending=False)



Similarity metric with Best Centroid Convergence : 


,Similarity metrics,Centroids Convergence,Previous SSE,New SSE,Max Iterations
1,Cosine SSE,2.546817e+10,2.541158e+10,2.542545e+10,2.541673e+10
2,Jaccard SSE,2.541523e+10,2.548318e+10,2.543418e+10,2.558791e+10
0,Euclidean SSE,2.532199e+10,2.531830e+10,2.549525e+10,2.540478e+10


In [19]:
print("Similarity metric with Best SSE : ")
newSSE_table.sort_values(by='New SSE', ascending=True)



Similarity metric with Best SSE : 


,Similarity metrics,Centroids Convergence,Previous SSE,New SSE,Max Iterations
1,Cosine SSE,2.546817e+10,2.541158e+10,2.542545e+10,2.541673e+10
2,Jaccard SSE,2.541523e+10,2.548318e+10,2.543418e+10,2.558791e+10
0,Euclidean SSE,2.532199e+10,2.531830e+10,2.549525e+10,2.540478e+10


In [20]:
print("Similarity metric with least iteration number : ")
newSSE_table.sort_values(by='Max Iterations', ascending=True)

Similarity metric with least iteration number : 


,Similarity metrics,Centroids Convergence,Previous SSE,New SSE,Max Iterations
0,Euclidean SSE,2.532199e+10,2.531830e+10,2.549525e+10,2.540478e+10
1,Cosine SSE,2.546817e+10,2.541158e+10,2.542545e+10,2.541673e+10
2,Jaccard SSE,2.541523e+10,2.548318e+10,2.543418e+10,2.558791e+10


# Compare SSE wrt stop conditions

Based on the results obtained during testing:
- Euclidean SSEs wrt stop criteria: 
    - Centroids converge:  25429959337.949955
    - New SSE is larger than previous SSE:  25407016682.77918
    - Reached max number of iters (100):  25321974749.58356
- Cosine SSEs wrt stop criteria: 
    - Centroids converge:  25555029379.927616
    - New SSE is larger than previous SSE:  25610084771.580967
    - Reached max number of iters (100):  25561511207.494164
- jaccard SSEs wrt stop criteria: 
    - Centroids converge:  25414464925.44861
    - New SSE is larger than previous SSE:  25446594079.129356
    - Reached max number of iters (100):  25451185367.17009
    
The best models determined by SSE wrt to the different stop conditions:
- Centroids converge: jaccard
- New SSE is larger than previous SSE: Euclidean
- Reached max number of iters (100): Euclidean

Thus the best overall K_Means model as found from this analysis is Euclidean

# Q5
## Concluding observations and algorithm analysis

In conclusion to this algorithmic analysis it would seem that overall the most consistent and reliable dist metric for the K_Means model is difficult to determine. Each of these metrics had tests that they appeared to be the best at; euclidean dist did the best in regard to the SSE metric both overall and with respect to the majority of the stop conditions, meanwhile jaccard had the best accuracy score when compared to the label set while euclidean had the lowest accuracy score. The cosine dist metric seems to be a sort of compensation between the two with an overall SSE better than jaccard but worse than euclidean and accuracy being better than euclidean but worse than jaccard.

In the future other dist metrics such as manhattan dist will have to be tested on the data set to see if any better performance can be obtained. Other methods than the one used here for determining the initial clusters will likely also help improve the performance of this model since initial centroids could be selected in strategic poss rather than just using a random point in the data set.